### 네이버 웹툰 1개의 회차에 대한 Image 다운로드

In [ ]:
import requests
from bs4 import BeautifulSoup
import os


def download_one_episode(title, no, url):
    req_header = {
        "referer": url,
        "User-Agent": "Mozilla/5.0",  # UA 추가 (웹툰 서버 보호용)
    }
    res = requests.get(url,headers=req_header)

    if res.ok:
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')

        #이미지 src 추출
        img_tags = soup.select("img[src*='_IMAG01']")
        img_url_list = [img['src'] for img in img_tags if img.has_attr('src')]

        print(f"img_tags 수: {len(img_tags)}") 
        imgdir_name = os.path.join('img', title, str(no))
        if not os.path.isdir(imgdir_name):
            os.makedirs(imgdir_name)  # 하위경로까지 생성

        for img_url in img_url_list:
            img_res = requests.get(img_url, headers=req_header)
            if img_res.ok:
                img = img_res.content
                file_name = os.path.basename(img_url)
                file_path = os.path.join(imgdir_name, file_name)

                with open(file_path, "wb") as file:
                    file.write(img)
                print(f"Writing to {file_path}")
            else:
                print(f"Error Code = {res.status_code}")

    else:
        print(f"Error Code = {res.status_code}")


download_one_episode(
    '쌉초의난',
    '92',
    'https://comic.naver.com/webtoon/detail?titleId=823195&no=92&week=wed'
)

In [ ]:
download_one_episode(
    '쌉초의난',
    '93',
    'https://comic.naver.com/webtoon/detail?titleId=823195&no=93&week=wed'
)

#### 다른 코드로 풀어봄


In [ ]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

def download_one_episode(title,no,url) :
    req_header = {'referer':url}
    res= requests.get(url)
    
    if res.ok :
        soup = BeautifulSoup(res.text,'html.parser')
        
        img_tags = soup.select("img[src*='IMAG01']")
        imgurl_list = [img['src'] for img in img_tags]
        
        if img_tags :
            save_dir = Path('img')/title /str(no)
            save_dir.mkdir(parents=True, exist_ok=True)
        
        for idx,img_url in enumerate(imgurl_list,1) :
            res=requests.get(img_url,headers=req_header)
            res.raise_for_status()
            
            file_name = Path(img_url).name
            save_path = save_dir / file_name
            save_path.write_bytes(res.content)
            print(f'다운로드 완료: {save_path} ({save_path.stat().st_size:,} bytes)')
            
            
download_one_episode('쌉초의난',94,'https://comic.naver.com/webtoon/detail?titleId=823195&no=94&week=wed')

### 하나의 웹툰과 여러 개의 회차에 대한 Image 다운로드

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]

    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}&page=1'
               #https://comic.naver.com/webtoon/detail?titleId=826419&no=46
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok:
        #pprint(res.json()['articleList'])
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('대학일기','https://comic.naver.com/webtoon/list?titleId=679519')

In [ ]:
from urllib.parse import urlparse, parse_qs

"""url에 titleId를 반환하는 함수"""
def get_title_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    return title_id

#테스트 하기
url = 'https://comic.naver.com/webtoon/list?titleId=679519'
print(get_title_id(url))

In [ ]:
def calculate_pages(total_items, items_per_page=20):
    """총 페이지 수 계산 함수"""
    return (total_items + items_per_page - 1) // items_per_page

# 예제 사용
total_items = 49
items_per_page = 20

total_pages = calculate_pages(total_items, items_per_page)
print(f"총 {total_items}개의 항목을 {items_per_page}개씩 출력할 때 필요한 페이지 수: {total_pages}")
# 출력: 총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3

### 하나의 웹툰과 여러 개의 회차에 대한 Image 다운로드 2

In [ ]:
import requests

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        #for count in range(calculate_pages(total_count)):
        page = 1 
        req_param = { "page": page}
        print(req_param)
        res = requests.get(ajax_url, params=req_param)
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
            
            download_one_episode(title,no,detail_url)

if __name__ == '__main__': 
    download_all_episode('대학일기','https://comic.naver.com/webtoon/list?titleId=679519')